<a href="https://colab.research.google.com/github/Mova-2020/Subworded-Explanatory-Dictionary-on-Physics-/blob/main/Term_comparison_v07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython import get_ipython
from IPython.display import display

!pip install gensim
!pip install -U sentence-transformers
!pip install spacy
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
!python -m spacy download uk_core_news_sm
import spacy
from sentence_transformers import SentenceTransformer
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import ngrams # Import ngrams
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 76.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('uk_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
model = SentenceTransformer('BAAI/bge-m3')
# Load a pre-trained sentence transformer model

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Helper functions for each criterion

In [3]:
# 1. Exactness (cosine similarity between term and definition)
def exactness(term, definition):
    # Calculates the semantic similarity between a term and its definition using SentenceTransformer.

    # Encode term and definition into sentence embeddings using the loaded model
    term_embedding = model.encode(term)
    definition_embedding = model.encode(definition)

    # Calculate cosine similarity
    cosine_sim = cosine_similarity(term_embedding.reshape(1, -1), definition_embedding.reshape(1, -1))[0][0]

    return cosine_sim

In [4]:
# 2. Essentiality (ratio of largest and second-largest entailment degrees from definition)
def essentiality(entailments):
    largest = max(entailments)
    second_largest = sorted(entailments, reverse=True)[1]
    return largest / second_largest if second_largest != 0 else float('inf')

In [5]:
# 3. Plainness (ratio of subwords in the term and its definition)
def plainness(term, definition):
    # Determine subword length dynamically (e.g., average word length)
    avg_word_len_term = sum(len(word) for word in term.split()) / len(term.split()) if term.split() else 0
    avg_word_len_def = sum(len(word) for word in definition.split()) / len(definition.split()) if definition.split() else 0
    subword_len = int(round((avg_word_len_term + avg_word_len_def) / 2)) or 1  # Default to 1 if average is 0

    # Generate subwords (all consecutive sequences of characters within a word)
    term_subwords = set()
    for word in term.split():
        for i in range(len(word) - subword_len + 1):
            term_subwords.add(word[i:i + subword_len])

    definition_subwords = set()
    for word in definition.split():
        for i in range(len(word) - subword_len + 1):
            definition_subwords.add(word[i:i + subword_len])

    common_subwords = term_subwords.intersection(definition_subwords)
    return len(common_subwords) / len(term_subwords) if term_subwords else 0

In [7]:
# 4. Derivativity (favorable word structure and ending)
def derivativity(term):
    bad_endings = ("nnja", "tja", "ння", "тя")
    num_words = len(term.replace('-', ' ').split())  # Рахуємо слова, розділені пробілами або дефісами

    if term.endswith(bad_endings):
        return 0  # Bad derivation if ending in "nnja", "tja", "ння", "тя"

    if num_words == 1:
        return 1  # Просте слово без закінчень "nnja", "tja", "ння", "тя"
    elif num_words == 2:
        return 0.5  # Два слова без закінчень "nnja", "tja", "ння", "тя"
    else:
        return 0  # Три і більше слова


In [8]:
# 5. Good sound (phonotactic rules compliance)
def good_sound(term):
    bad_clusters = ["ngh", "shr", "zhr", "r r", "stj st", "st st", "нг", "шр", "жр", "р р", "сть ст", "ст ст"]
    if any(cluster in term for cluster in bad_clusters):
        return 0
    # Розбиваємо термін на слова
    words = term.split()

    # Перевіряємо наявність нових кластерів між словами
    for i in range(len(words) - 1):
        if words[i].endswith("р ") and words[i+1].startswith("р"):
            return 0
        if words[i].endswith("сть ") and words[i+1].startswith("ст"):
            return 0
    return 1 # Return 1 if no bad clusters are found

In [9]:
# 6. Systemic feature (existence of hypernyms or hyponyms in the dictionary)
def systemic_feature(term, dictionary_entries):
    related_entries = [entry for entry in dictionary_entries if term in entry]
    return len(related_entries)

In [10]:
# 7. Organicity (inverse of maximum subword length)
def organicity(term):
    subwords = term.split()
    max_length = max(len(subword) for subword in subwords)
    return 1 / max_length if max_length != 0 else 0

In [11]:
# 8. Compatibility (syntactic valence of the term)
def compatibility(term, term_valences):
    # Calculates the syntactic valence (number of POS connections) of the term.

    # Load Ukrainian spaCy model (if not already loaded)
    try:
        if 'nlp_compatibility' not in globals():
            global nlp_compatibility
            nlp_compatibility = spacy.load("uk_core_news_sm")
    except NameError:
        nlp_compatibility = spacy.load("uk_core_news_sm")

    # Analyze the term in isolation to consider broader syntactic possibilities
    doc = nlp_compatibility(term)

    # Find the main verb or noun in the term (if any)
    main_token = next((token for token in doc if token.pos_ in ("VERB", "NOUN")), None)

    if main_token:
        # Calculate valence as the number of unique POS tags of children and head
        surrounding_pos = [token.pos_ for token in main_token.children]

        # Include the POS tag of the head token (if it exists)
        if main_token.head:
            surrounding_pos.append(main_token.head.pos_)

        valence = len(set(surrounding_pos))
    else:
        valence = 0  # Default valence if no main verb or noun is found

    return valence

In [12]:
# 9. Unambiguity (inverse of number of definitions for the term)
def unambiguity(definitions_count):
    return 1 / definitions_count if definitions_count != 0 else 0

In [13]:
# 10. Nominativity (absence of descriptive structures with prepositions)
def nominativity(term):
    prepositions = ["z ", "zi ", "iz ", "za ", "proty ", "po ", "vid ", "do ", "з ", "зі ", "із ", "за ", "проти ", "по ", "від ", "до "]
    n_conj = sum(1 for prep in prepositions if prep in term)  # Count prepositions in the term
    return 1 / (1 + n_conj)  # Calculate nominativity according to the formula

In [14]:
# 11. Brevity (inverse number of symbols in the term)
def brevity(term):
    return 1 / len(term) if len(term) != 0 else 0

Data input

In [15]:
def get_terms_from_user():
    # Просить користувача ввести терміни та їх означення для порівняння
    while True:
        try:
            num_terms_input = input("Введіть кількість термінів для порівняння (за замовчуванням 2): ")
            num_terms = int(num_terms_input) if num_terms_input else 2  # Use 2 as default if input is empty
            break
        except ValueError:
            print("Неправильний ввід. Будь ласка, введіть число.")

    terms_and_definitions = {}  # Using a dictionary to store terms and definitions
    for i in range(num_terms):
        term = input(f"Введіть термін {i + 1}: ")
        definition = input(f"Введіть означення для терміна '{term}': ")
        terms_and_definitions[term] = {'definition': definition, 'systemic_feature': 0, 'definitions_count': 0}  # Store term, definition, initialize systemic_feature and definitions_count

    # Now ask about systemic features and definitions count after getting all terms and definitions
    for term, data in terms_and_definitions.items():
        while True:
            systemic_input = input(f"Чи наявні в терміносистемі гіпероніми або гіпоніми до терміна '{term}'?\nВведіть '1', якщо такі є, або '0', якщо таких немає: ")
            if systemic_input in ["1", "0"]:
                data['systemic_feature'] = int(systemic_input) # Update systemic_feature in the dictionary
                break
            else:
                print("Неправильний ввід. Будь ласка, введіть 1 або 0.")

        while True:
            definitions_count_input = input(f"Скільки значень відповідно до різних означень має термін '{term}'?: ")
            try:
                definitions_count = int(definitions_count_input)
                data['definitions_count'] = definitions_count # Update definitions_count in the dictionary
                break
            except ValueError:
                print("Неправильний ввід. Будь ласка, введіть число.")

    return terms_and_definitions  # Return the dictionary

Scores

In [16]:
def calculate_scores_for_terms(terms_and_definitions):
    # Обчислює бали для кожного терміна

    # Load Ukrainian spaCy model
    nlp = spacy.load("uk_core_news_sm")

    # Assign valence (number of POS connections)
    term_valences = {}
    for term, data in terms_and_definitions.items():
        definition = data['definition'] # Access definition from the nested dictionary
        # Create a sentence with the term and definition for context
        sentence = f"{term} {definition}"
        doc = nlp(sentence)

        # Find the term in the parsed sentence
        term_token = next((token for token in doc if token.text == term), None)

        if term_token:
            # Get POS tags of surrounding words
            surrounding_pos = [token.pos_ for token in term_token.children]

            # Calculate valence as the number of unique POS tags
            valence = len(set(surrounding_pos))
        else:
            valence = 0  # Default valence if term not found

        term_valences[term] = valence

    scores = {}
    for term, data in terms_and_definitions.items():  # Iterate through the nested dictionary
        definition = data["definition"]
        systemic_feature_value = data["systemic_feature"]
        definitions_count = data["definitions_count"]  # Get definitions_count

        # Calculate Exactness using cosine similarity
        exactness_score = exactness(term, definition)
        print(f"Cosine similarity for '{term}': {exactness_score}") # Print for debugging

        # Essentiality Calculation (placeholder - requires entailment logic)
        # Replace this with your actual entailment calculation
        entailments = [0.8, 0.6, 0.4]  # Example entailment degrees
        essentiality_score = essentiality(entailments)

        # Plainness calculation using NLTK
        plainness_score = plainness(term, definition)


        # Initialize scores for the current term
        scores_term = {
            "Exactness": exactness_score,
            "Essentiality": essentiality_score,
            "Plainness": plainness_score,
            "Derivativity": derivativity(term),
            "Good Sound": good_sound(term),
            # Use the populated dictionary
            "Systemic Feature": systemic_feature_value,  # Access systemic feature value
            "Organicity": organicity(term),
            # Use the populated term_valences
            "Compatibility": compatibility(term, term_valences),
            "Unambiguity": unambiguity(definitions_count),  # Use definitions_count for Unambiguity
            "Nominativity": nominativity(term),  # No prepositionss in the term
            "Brevity": brevity(term),
        }
        scores[term] = scores_term
        #Assign scores_term to scores dictionary with term as key
    return scores


Comparison

In [25]:
terms_and_definitions = get_terms_from_user() # Get terms and definitions first
scores = calculate_scores_for_terms(terms_and_definitions) # Then calculate scores

Введіть кількість термінів для порівняння (за замовчуванням 2): 7
Введіть термін 1: зелений
Введіть означення для терміна 'зелений': Який має забарвлення одного з основних кольорів спектру – середній між жовтим і блакитним; який має колір свіжої трави, листя, зелені.
Введіть термін 2: червоний
Введіть означення для терміна 'червоний': Який має забарвлення одного з основних кольорів спектру – середній між жовтим і блакитним; який має колір свіжої трави, листя, зелені.
Введіть термін 3: синій
Введіть означення для терміна 'синій': Який має забарвлення одного з основних кольорів спектру – середній між жовтим і блакитним; який має колір свіжої трави, листя, зелені.
Введіть термін 4: жовтий
Введіть означення для терміна 'жовтий': Який має забарвлення одного з основних кольорів спектру – середній між жовтим і блакитним; який має колір свіжої трави, листя, зелені.
Введіть термін 5: говорити
Введіть означення для терміна 'говорити': Який має забарвлення одного з основних кольорів спектру – сер

In [26]:
for term, term_scores in scores.items():
    print(f"Бали для терміна '{term}': {term_scores}")
    overall_score = sum(term_scores.values()) / len(term_scores)
    print(f"Загальний бал для терміна '{term}': {overall_score}")

Бали для терміна 'зелений': {'Exactness': 0.48752457, 'Essentiality': 1.3333333333333335, 'Plainness': 0.0, 'Derivativity': 1, 'Good Sound': 1, 'Systemic Feature': 1, 'Organicity': 0.14285714285714285, 'Compatibility': 0, 'Unambiguity': 0.125, 'Nominativity': 1.0, 'Brevity': 0.14285714285714285}
Загальний бал для терміна 'зелений': 0.566506562552927
Бали для терміна 'червоний': {'Exactness': 0.44276077, 'Essentiality': 1.3333333333333335, 'Plainness': 0.0, 'Derivativity': 1, 'Good Sound': 1, 'Systemic Feature': 1, 'Organicity': 0.125, 'Compatibility': 0, 'Unambiguity': 0.25, 'Nominativity': 1.0, 'Brevity': 0.125}
Загальний бал для терміна 'червоний': 0.5705540089896232
Бали для терміна 'синій': {'Exactness': 0.411471, 'Essentiality': 1.3333333333333335, 'Plainness': 0.0, 'Derivativity': 1, 'Good Sound': 1, 'Systemic Feature': 1, 'Organicity': 0.2, 'Compatibility': 0, 'Unambiguity': 0.3333333333333333, 'Nominativity': 1.0, 'Brevity': 0.2}
Загальний бал для терміна 'синій': 0.58892160690